<a href="https://www.kaggle.com/code/tmishinev/world-population-eda-map-animations?scriptVersionId=105239203" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style='color:#203354;border: 2px outset #203354;;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:30px;font-weight:bold;text-align:center;'>
    World Population Animation Maps
</div>

#### [1. Load and Transform the Dataset](#load)

#### [2. Animation Maps](#location)
 - [2.1 Population per Country per Country](#number)
 - [2.2 Density per Country](#density)
 - [2.3 Percent of Total World Population](#percent)
 
#### [3. Big Movers](#movers)
 - [3.1 Countries - Increasing Population (TOP 20)](#pos_growth)
 - [3.2 Countries - Declining Population](#neg_growth)

#### [4. Continent Population Trends](#continent)
 - [4.1 Continent Population Growth](#cont_population)
 - [4.2 Continent Density Growth](#cont_density)

In [1]:
#libraries
import pandas as pd
import numpy as np
import plotly.express as px

BGCOLOR = '#eeeeee'
WIDTH = 800

<a id = 'load'></a> 
## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:20px;font-weight:bold;text-align:left;'>1. Load and Transform the Dataset</div>

In [2]:
# Load the Dataset
df = pd.read_csv('../input/world-population-dataset/world_population.csv')
df.head(3)

,Rank,CCA3,Country,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,36,AFG,Afghanistan,Kabul,Asia,41128771,38972230,33753499,28189672,19542982,10694796,12486631,10752971,652230,63.0587,1.0257,0.52
1,138,ALB,Albania,Tirana,Europe,2842321,2866849,2882481,2913399,3182021,3295066,2941651,2324731,28748,98.8702,0.9957,0.04
2,34,DZA,Algeria,Algiers,Africa,44903225,43451666,39543154,35856344,30774621,25518074,18739378,13795915,2381741,18.8531,1.0164,0.56


In [3]:
#melt the dataset 
df_year_population = df.loc[:, ['CCA3', 'Country', 'Area (km²)', 'Continent', '1970 Population', '1980 Population', 
                                '1990 Population', '2000 Population','2010 Population','2015 Population','2020 Population','2022 Population']]

df_year_population = df_year_population.melt(id_vars = ['CCA3', 'Country','Area (km²)', 'Continent'], value_vars = ['1970 Population', '1980 Population', 
                                '1990 Population', '2000 Population','2010 Population','2015 Population','2020 Population','2022 Population'])

df_year_population['variable'] = df_year_population['variable'].apply(lambda x: int(x[:4]))
df_year_population['density'] = round(df_year_population['value']/df_year_population['Area (km²)'],2)
df_year_population.rename(columns = {'variable':'year', 'value': 'population' }, inplace = True)

#add percentage of all population
df_year_total = df_year_population.groupby('year')['population'].sum().reset_index()
df_year_total.rename(columns = {'population': 'total_population' }, inplace = True)

df_year_population = df_year_population.merge(df_year_total, on = 'year')
df_year_population['percent_of_total'] = round(df_year_population['population']/df_year_population['total_population']*100,2)
                                
df_year_population.tail(3)

,CCA3,Country,Area (km²),Continent,year,population,density,total_population,percent_of_total
1869,YEM,Yemen,527968,Asia,2022,33696614,63.82,7973413042,0.42
1870,ZMB,Zambia,752612,Africa,2022,20017675,26.60,7973413042,0.25
1871,ZWE,Zimbabwe,390757,Africa,2022,16320537,41.77,7973413042,0.20


<a id = 'maps'></a>

## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:20px;font-weight:bold;text-align:left;'>2. World Population Animation Maps</div>

<a id = 'number'></a>
## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:18px;font-weight:bold;text-align:left;'>2.1 Population per Country</div>

In [4]:
# choropleth by number
px.choropleth(df_year_population, 
              locations = 'CCA3',
              color=df_year_population["population"], 
              animation_frame="year",
              color_continuous_scale = [[0, '#203354'],
                      [0.05, '#bbbbbb'],
                      [0.7, '#751c35'],
                      [1, '#751c35'],
                     ],
              range_color=(0, 1500000000),
              title='World Population Number Animation',
              height=800

             )

<a id = 'density'></a>
## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:18px;font-weight:bold;text-align:left;'>2.2 Density per Country</div>

In [5]:
# choropleth by number
px.choropleth(df_year_population, 
              locations = 'CCA3',
              color=df_year_population["density"], 
              animation_frame="year",
              color_continuous_scale = [[0, '#203354'],
                      [0.05, '#bbbbbb'],
                      [0.7, '#751c35'],
                      [1, '#751c35'],
                     ],
              range_color=(0, np.percentile(df_year_population["density"],90)),
              title='World Population Density Animation',
              height=800

             )

<a id = 'percent'></a>
## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:18px;font-weight:bold;text-align:left;'>2.3 Percent of Total World Population</div>

In [6]:
# choropleth by number
px.choropleth(df_year_population, 
              locations = 'CCA3',
              color=df_year_population["percent_of_total"], 
              animation_frame="year",
              color_continuous_scale = [[0, '#203354'],
                                        [0.02, '#203354'],
                      [0.1, '#bbbbbb'],
                      [0.8, '#751c35'],
                      [1, '#751c35'],
                     ],
              range_color=(0, 25),
              title='Percent of Total World Population',
              height=800

             )

<a id = 'movers'></a>
## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:20px;font-weight:bold;text-align:left;'>3. Big Movers</div>

<a id = 'country_growth'></a>
## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:18px;font-weight:bold;text-align:left;'>3.1 Positive Growth (TOP 20)</div>

In [7]:
df_delta = df.copy()
df_delta['delta'] = df_delta['2022 Population'] - df_delta['1970 Population']
df_delta.sort_values('delta', ascending = False, inplace = True)
df_delta.head(5)

,Rank,CCA3,Country,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,delta
92,2,IND,India,New Delhi,Asia,1417173173,1396387127,1322866505,1240613620,1059633675,870452165,696828385,557501301,3287590,431.0675,1.0068,17.77,859671872
41,1,CHN,China,Beijing,Asia,1425887337,1424929781,1393715448,1348191368,1264099069,1153704252,982372466,822534450,9706961,146.8933,1.0000,17.88,603352887
156,5,PAK,Pakistan,Islamabad,Asia,235824862,227196741,210969298,194454498,154369924,115414069,80624057,59290872,881912,267.4018,1.0191,2.96,176533990
149,6,NGA,Nigeria,Abuja,Africa,218541212,208327405,183995785,160952853,122851984,95214257,72951439,55569264,923768,236.5759,1.0241,2.74,162971948
93,4,IDN,Indonesia,Jakarta,Asia,275501339,271857970,259091970,244016173,214072421,182159874,148177096,115228394,1904569,144.6529,1.0064,3.45,160272945


In [8]:
pos_growth = df_delta.head(20)
pos_growth

df_positive = df_year_population.loc[df_year_population['Country'].isin(pos_growth['Country']), :]
# choropleth by number
fig_map = px.choropleth(df_positive, 
              locations = 'CCA3',
              color=df_positive["population"], 
              animation_frame="year",
              color_continuous_scale = [[0, '#203354'],
                      [0.05, '#bbbbbb'],
                      [0.7, '#751c35'],
                      [1, '#751c35'],
                     ],
              range_color=(df_positive.population.min(), df_positive.population.max()),
              title='TOP 20 Countries Increasing Population',
              height=800

             )
fig_map.show()

fig_bar = px.bar(df_delta.head(20), x = 'Country', y = 'delta', color_discrete_sequence = ['#203354', '#bbbbbb', '#751c35', '#101010', '#108810', '#1010dd'],
                title = 'Population Delta 1970 to 2022')
fig_bar.update_xaxes(tickangle = 90)
fig_bar.show()

<a id = 'neg_growth'></a>

## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:18px;font-weight:bold;text-align:left;'>3.2 Negative Growth</div>

In [9]:
neg_growth = df_delta.loc[df_delta['delta'] < 0, :]
neg_growth

,Rank,CCA3,Country,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,delta
226,234,VAT,Vatican City,Vatican City,Europe,510,520,564,596,651,700,733,752,1,510.0000,0.9980,0.00,-242
150,232,NIU,Niue,Alofi,Oceania,1934,1942,1847,1812,2074,2533,3637,5185,260,7.4385,0.9985,0.00,-3251
44,223,COK,Cook Islands,Avarua,Oceania,17011,17029,17695,17212,15897,17123,17651,20470,236,72.0805,1.0005,0.00,-3459
137,230,MSR,Montserrat,Brades,North America,4390,4500,5059,4938,5138,10805,11452,11402,102,43.0392,0.9939,0.00,-7012
61,156,EST,Estonia,Tallinn,Europe,1326062,1329444,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.9980,0.02,-35937
170,64,ROU,Romania,Bucharest,Europe,19659267,19442038,19906079,20335211,21919876,22836234,22125224,19922618,238391,82.4665,1.0171,0.25,-263351
90,94,HUN,Hungary,Budapest,Europe,9967308,9750573,9844246,9986825,10202055,10375989,10698679,10315366,93028,107.1431,1.0265,0.12,-348058
133,135,MDA,Moldova,Chisinau,Europe,3272996,3084847,3277388,3678186,4251573,4480199,4103240,3711140,33846,96.7026,1.0691,0.04,-438144
117,141,LTU,Lithuania,Vilnius,Europe,2750055,2820267,2963765,3139019,3599637,3785847,3521206,3210147,65300,42.1142,0.9869,0.03,-460092
46,130,HRV,Croatia,Zagreb,Europe,4030358,4096868,4254815,4368682,4548434,4873707,4680144,4492638,56594,71.2153,0.9927,0.05,-462280


In [10]:
df_negative = df_year_population.loc[df_year_population['Country'].isin(neg_growth['Country']), :]
# choropleth by number
fig_map = px.choropleth(df_negative, 
              locations = 'CCA3',
              color=df_negative["population"], 
              animation_frame="year",
              color_continuous_scale = [[0, '#203354'],
                      [0.05, '#bbbbbb'],
                      [0.7, '#751c35'],
                      [1, '#751c35'],
                     ],
              range_color=(df_negative.population.min(), df_negative.population.max()),
              title='Countries Declining Population',
              height=800

             )

fig_map.show()

fig_bar = px.bar(neg_growth.sort_values('delta'), x = 'Country', y = 'delta', color_discrete_sequence = ['#203354', '#bbbbbb', '#751c35', '#101010', '#108810', '#1010dd'],
                title = 'Population Delta 1970 to 2022')
fig_bar.update_xaxes(tickangle = 90)
fig_bar.show()

<a id = 'trends'></a>
## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:20px;font-weight:bold;text-align:left;'>4. Continent Trends</div>

<a id = 'cont_population'></a>
## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:18px;font-weight:bold;text-align:left;'>4.1 Continent Populaton Growth</div>

In [11]:
df_cont = df_year_population.groupby(['year', 'Continent'])['population'].sum().reset_index()
fig = px.bar(df_cont, y = 'Continent',x = 'population',animation_frame = 'year',
              color_discrete_sequence = ['#751c35', '#bbbbbb', '#751c35', '#101010', '#108810', '#1010dd'])
fig.update_xaxes(
    range=(0, 5e9),
    constrain='domain'
)

fig.update_layout(width = WIDTH, paper_bgcolor = BGCOLOR, plot_bgcolor = BGCOLOR, title = 'Population Growth by Continent')

fig.show()

<a id = 'cont_density'></a>
## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:18px;font-weight:bold;text-align:left;'>4.2 Continent Density Growth</div>

In [12]:
df_cont_density = df_year_population.groupby(['year', 'Continent'])[['population', 'Area (km²)']].sum().reset_index()
df_cont_density['density'] = round(df_cont_density['population']/df_cont_density['Area (km²)'],1)

df_cont = df_cont_density.groupby(['year', 'Continent'])['density'].sum().reset_index()
fig = px.bar(df_cont, y = 'Continent',x = 'density',animation_frame = 'year',
              color_discrete_sequence = ['#751c35', '#bbbbbb', '#751c35', '#101010', '#108810', '#1010dd'])
fig.update_xaxes(
    range=(0, 200),
    constrain='domain'
)



fig.update_layout(width = WIDTH, paper_bgcolor = BGCOLOR, plot_bgcolor = BGCOLOR, title = 'Density Growth by Continent [ppl/Area[km^2]]')
fig.show()

## <div style='color:#203354;background-color:rgba(222, 222, 222, 0.85);padding:10px 10px;font-size:18px;font-weight:bold;text-align:left;'>Thanks and upvote if you like it :)</div>